# <a name="top">Exercise 2: HGNC and MySQL</span>

## Table of content
0. <a href="#import_and_preparations">Imports and preparations</a>
0. <a href="#problem">Defining the problem</a>
0. <a href="#objective">Objective</a>
0. <a href="#tasks_in_general">Tasks in general</a>
0. <a href="#tasks">Tasks</a>
0. <a href="#problem">Defining the problem</a>

## <a id="import_and_preparations">Imports and preparations</a>

In [1]:
import pandas as pd
import os , json, pymysql, re
base = os.path.join(os.environ['BUG_FREE_EUREKA_BASE'])
%matplotlib inline

In [2]:
pathToFile = os.path.join(base,'data','exercise05','nodes_in_Alzheimer_BEL_disease_network.tsv')
df = pd.read_table(pathToFile)

## <a id="problem">Defining the problem</a>
We have already loaded a list of biological entities in the context of Alzheimer's disease in a pandas `DataFrame`. The column **`value`** identifies in this `DataFrame` biological entities in specific namespace (stored in the column `namespace`). **`namespace`** and **`value`** together are a ***unique identifier*** for the entities. 

One of the most used namespaces in our dataset is HGNC. The HGNC database is maintained by the HUGO  Gene Nomenclature Committee and stores approved gene names, symbols (short-form abbreviation, column our `value`) and addtional information about genes.

OK, even we have now a clear identify with the short name in our `DataFrame`: But do you know the recommended names of these short names, to which gene family they belong or the gene ID from NCBI?

In [3]:
list(df[(df.namespace=='HGNC')].value[:10])

['BACE1',
 'APP',
 'CLSTN1',
 'KLC1',
 'PSEN1',
 'NCSTN',
 'APH1A',
 'PSEN2',
 'ARRB2',
 'GPR3']

 Ice cream for all if you can give me all recommended names (without any further search)!

## <a id="objective">Objective</a>
+ Find new insights into Alzheimer's disease by analysing the information about genes

## <a id="tasks_in_general">Tasks in general</a>

   0. Analyse the JSON file, find a way to automatically scan the whole json for datatypes
   0. Connect to MySQL, create database
   0. Create appropriate tables in MySQL database
   0. Store the data in the database
   0. Find the gene family with the highest frequency 
       + by combining 
           + pandas dataframe
           + MySQL database
       + loading database into dataframe
       + loading dataframe into database
       
links:
+ [Reference manual](http://dev.mysql.com/doc/refman/5.7/en/)
+ [HUGO Gene Nomenclature Committee (HGNC)](http://www.genenames.org/)

## <a id="tasks">Tasks</a>

### Load the JSON file

The data is in the JSON format - this means that all atomic data is in nested dictionaries and lists.

In [4]:
data_path = os.path.join(base,'data', 'exercise02', 'hgnc_complete_set.json')

with open(data_path) as f:
    hgnc_json = json.load(f)

### Exploration of data structure

We want to understand what data is inside this json. To do this, we'll look at the keys of each nested dictionary.

The first set of keys shows that there is a response (the data) and a response header (data about the way it was downloaded). We will further explore the response.

In [5]:
hgnc_json.keys()

dict_keys(['response', 'responseHeader'])

The response contains `numFound`, which lists how many results there are in `docs`. Disregard `start`.

In [6]:
hgnc_json['response'].keys()

dict_keys(['numFound', 'docs', 'start'])

In [7]:
hgnc_json['response']['docs'][0] # try wz .keys() # it is 1 to 1 RS and 1 to many RS

{'_version_': 1546503090507612160,
 'ccds_id': ['CCDS12976'],
 'cosmic': 'A1BG',
 'date_approved_reserved': '1989-06-30',
 'date_modified': '2015-07-13',
 'ensembl_gene_id': 'ENSG00000121410',
 'entrez_id': '1',
 'gene_family': ['Immunoglobulin like domain containing'],
 'gene_family_id': [594],
 'hgnc_id': 'HGNC:5',
 'location': '19q13.43',
 'location_sortable': '19q13.43',
 'locus_group': 'protein-coding gene',
 'locus_type': 'gene with protein product',
 'merops': 'I43.950',
 'mgd_id': ['MGI:2152878'],
 'name': 'alpha-1-B glycoprotein',
 'omim_id': [138670],
 'pubmed_id': [2591067],
 'refseq_accession': ['NM_130786'],
 'rgd_id': ['RGD:69417'],
 'status': 'Approved',
 'symbol': 'A1BG',
 'ucsc_id': 'uc002qsd.5',
 'uniprot_ids': ['P04217'],
 'uuid': 'c5fd27c5-7aa4-447c-83b0-1ccc73d90925',
 'vega_id': 'OTTHUMG00000183507'}

`docs` is a list where each entry is the data associated with a gene. Below is an example of the first element of this list:

In [8]:
print(json.dumps(hgnc_json['response']['docs'][0], indent=5)) # to be nicely formated


{
     "uniprot_ids": [
          "P04217"
     ],
     "ccds_id": [
          "CCDS12976"
     ],
     "locus_type": "gene with protein product",
     "date_modified": "2015-07-13",
     "refseq_accession": [
          "NM_130786"
     ],
     "ucsc_id": "uc002qsd.5",
     "mgd_id": [
          "MGI:2152878"
     ],
     "gene_family": [
          "Immunoglobulin like domain containing"
     ],
     "merops": "I43.950",
     "_version_": 1546503090507612160,
     "location_sortable": "19q13.43",
     "entrez_id": "1",
     "locus_group": "protein-coding gene",
     "cosmic": "A1BG",
     "omim_id": [
          138670
     ],
     "ensembl_gene_id": "ENSG00000121410",
     "uuid": "c5fd27c5-7aa4-447c-83b0-1ccc73d90925",
     "hgnc_id": "HGNC:5",
     "gene_family_id": [
          594
     ],
     "symbol": "A1BG",
     "pubmed_id": [
          2591067
     ],
     "vega_id": "OTTHUMG00000183507",
     "name": "alpha-1-B glycoprotein",
     "date_approved_reserved": "1989-06-30",
     "

### Create a Database Schema

Analyze the structure of each entry. Depending on the data type, decide what sort of relation you need to store the data. For example, a list would correspond to a 1-to-many relationship, while an atomic would correspond to a 1-to-1.

Your goal is to connect to your database, build an appropriate schema, and upload the data from the hgnc_json file. 

Choose one 1-to-n relationship and some 1-to-1 for your schema.

In [9]:
secrets_path = os.path.join(base, 'secrets.json')

with open(secrets_path) as f:
    secrets = json.load(f)

print(json.dumps(secrets, indent=2))

{
  "test_db": {
    "host": "localhost",
    "db": "mysql",
    "user": "root",
    "password": "1234",
    "port": 3306
  }
}


In [10]:
pd.io.sql._SQL_TYPES['string']['mysql']='VARCHAR (255)' # fixes the problem of too short varchar fields

In [11]:
db_params = secrets['test_db']

conn = pymysql.connect(**db_params) # **

In [12]:
schema_sql = """
select 'YOUR CODE HERE'
"""

with conn.cursor(pymysql.cursors.DictCursor) as cursor:
    cursor.execute(schema_sql)
    print(*cursor.fetchone())

YOUR CODE HERE


## ToDos
+ create a database with the name `hgnc`
+ Analyse you data structure by iterating over the data and identify 
    + datatypes
    + relationship type (1 to 1, 1 to many)
+ use a new cursor  to execute the same statement over entry of `docs`.
+ follow the suggestions below 
+ Upload the Data

## Solution

## Create the database 

In [13]:
# create database hereconn = pymysql.Connect(**secrets['test_db']) # to use sql
cursor = conn.cursor() 

cursor.execute(' create database if not exists hgnc')



C:\Users\randa\Anaconda3\lib\site-packages\pymysql\cursors.py:166: Warning: (1007, "Can't create database 'hgnc'; database exists")
  result = self._query(query)


1

In [14]:
#check if table is created
cursor.execute("show tables")
cursor.fetchall()

(('columns_priv',),
 ('db',),
 ('dea',),
 ('engine_cost',),
 ('event',),
 ('func',),
 ('general_log',),
 ('gtid_executed',),
 ('help_category',),
 ('help_keyword',),
 ('help_relation',),
 ('help_topic',),
 ('innodb_index_stats',),
 ('innodb_table_stats',),
 ('ndb_binlog_index',),
 ('plugin',),
 ('proc',),
 ('procs_priv',),
 ('proxies_priv',),
 ('server_cost',),
 ('servers',),
 ('slave_master_info',),
 ('slave_relay_log_info',),
 ('slave_worker_info',),
 ('slow_log',),
 ('tables_priv',),
 ('time_zone',),
 ('time_zone_leap_second',),
 ('time_zone_name',),
 ('time_zone_transition',),
 ('time_zone_transition_type',),
 ('user',))

### Analyse you data structure by iterating over the data

In [15]:
docs = hgnc_json['response']['docs'] # my data
print(type(docs))

<class 'list'>


In [16]:
dataTypes={'column1':[int],'column2':[str,int]} 

 # if the json mix up data types
# ... write here your code ti fill up the dataTypes dictionary like dataType = {'column1':[int],'column2':[str,int],...}
# you will find only int strn list in HGNC 

In [17]:
df.describe()

C:\Users\randa\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,node_id,node_id.1,position
count,11927.000000,11927.000000,130.000000
mean,5964.000000,5964.000000,2595.946154
std,3443.172665,3443.172665,24114.045581
min,1.000000,1.000000,4.000000
25%,2982.500000,2982.500000,NaN
50%,5964.000000,5964.000000,NaN
75%,8945.500000,8945.500000,NaN
max,11927.000000,11927.000000,275341.000000


In [18]:
df.head(20) # checking rows 

,node_id,BEL,aminoacid_Code_1,aminoacid_Code_2,function,list,modification,namespace,node_id.1,p_modType,position,undefined_namespace,valid_NSV,value
0,1,"a(CHEBI:""gamma-secretase inhibitor"")",NaN,NaN,a,NaN,NaN,CHEBI,1,NaN,NaN,NaN,NaN,gamma-secretase inhibitor
1,2,"complex(SCOMP:""gamma Secretase Complex"")",NaN,NaN,complex,NaN,NaN,SCOMP,2,NaN,NaN,NaN,NaN,"""gamma Secretase Complex"""
2,3,"p(HGNC:""BACE1"")",NaN,NaN,p,NaN,NaN,HGNC,3,NaN,NaN,NaN,True,BACE1
3,4,"reaction(reactants(p(HGNC:APP)),products(a(ADO...",NaN,NaN,reaction,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN
4,5,"p(HGNC:""APP"")",NaN,NaN,p,NaN,NaN,HGNC,5,NaN,NaN,NaN,True,APP
5,6,"a(ADO:""Abeta_42"")",NaN,NaN,a,NaN,NaN,ADO,6,NaN,NaN,NaN,True,Abeta_42
6,7,"path(MESHD:""Alzheimer Disease"")",NaN,NaN,path,NaN,NaN,MESHD,7,NaN,NaN,NaN,True,Alzheimer Disease
7,8,"p(HGNC:""CLSTN1"")",NaN,NaN,p,NaN,NaN,HGNC,8,NaN,NaN,NaN,True,CLSTN1
8,9,"complex(p(HGNC:CLSTN1),p(HGNC:KLC1))",NaN,NaN,complex,True,NaN,NaN,9,NaN,NaN,NaN,NaN,NaN
9,10,"p(HGNC:""KLC1"")",NaN,NaN,p,NaN,NaN,HGNC,10,NaN,NaN,NaN,True,KLC1


In [19]:
df.set_index('node_id', inplace=True)#Let's set the index_id

In [20]:
df.columns #check that with columns

Index(['BEL', 'aminoacid_Code_1', 'aminoacid_Code_2', 'function', 'list',
       'modification', 'namespace', 'node_id.1', 'p_modType', 'position',
       'undefined_namespace', 'valid_NSV', 'value'],
      dtype='object')

In [21]:
# example Ö 
#mylist =[1,1,2,2,5,7,8,9]
#{x for x in mylist if x > 3}

In [22]:
df['namespace']

node_id
1            CHEBI
2            SCOMP
3             HGNC
4              NaN
5             HGNC
6              ADO
7            MESHD
8             HGNC
9              NaN
10            HGNC
11          MESHPP
12             NaN
13            HGNC
14            HGNC
15            HGNC
16            HGNC
17             NaN
18             ADO
19           CHEBI
20           CHEBI
21           CHEBI
22           CHEBI
23           CHEBI
24            HGNC
25            HGNC
26             NaN
27            HGNC
28            HGNC
29            HGNC
30            HGNC
           ...    
11898         HGNC
11899         NIFT
11900        dbSNP
11901         NIFT
11902         NIFT
11903         GOBP
11904         NIFT
11905        dbSNP
11906        dbSNP
11907         HGNC
11908         NIFT
11909         NIFT
11910          NaN
11911    UNDEFINED
11912         BRCO
11913          NaN
11914         HGNC
11915         NIFT
11916         NIFT
11917        dbSNP
11918         NIFT
1191

In [23]:
namespace_distribution = df.namespace.value_counts()
namespace_distribution # checking repititions

HGNC         7636
MGI          1079
RGD           636
GOBP          612
CHEBI         339
UNDEFINED     296
dbSNP         161
MESHD         124
SCHEM          79
SFAM           78
PMIBP          32
MESHPP         26
GOCC           25
PMICHEM        21
NIFT           20
ADO            15
PTS            14
SCOMP           9
BRCO            9
CHEBIID         5
PMIPFAM         5
MESHCS          4
CHEMBL          3
SDIS            3
PMIDIS          2
PMICOMP         2
Name: namespace, dtype: int64

Now we check if there could be more than one datatype per key. We can do this by iterating over the dictionary `dataTypes`. This time we have curly brackets surronding the list compression. This create a set.

In [24]:
df_short = pd.read_table(pathToFile, nrows=100)
df_short

,node_id,BEL,aminoacid_Code_1,aminoacid_Code_2,function,list,modification,namespace,node_id.1,p_modType,position,undefined_namespace,valid_NSV,value
0,1,"a(CHEBI:""gamma-secretase inhibitor"")",NaN,NaN,a,NaN,NaN,CHEBI,1,NaN,NaN,NaN,NaN,gamma-secretase inhibitor
1,2,"complex(SCOMP:""gamma Secretase Complex"")",NaN,NaN,complex,NaN,NaN,SCOMP,2,NaN,NaN,NaN,NaN,"""gamma Secretase Complex"""
2,3,"p(HGNC:""BACE1"")",NaN,NaN,p,NaN,NaN,HGNC,3,NaN,NaN,NaN,True,BACE1
3,4,"reaction(reactants(p(HGNC:APP)),products(a(ADO...",NaN,NaN,reaction,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN
4,5,"p(HGNC:""APP"")",NaN,NaN,p,NaN,NaN,HGNC,5,NaN,NaN,NaN,True,APP
5,6,"a(ADO:""Abeta_42"")",NaN,NaN,a,NaN,NaN,ADO,6,NaN,NaN,NaN,True,Abeta_42
6,7,"path(MESHD:""Alzheimer Disease"")",NaN,NaN,path,NaN,NaN,MESHD,7,NaN,NaN,NaN,True,Alzheimer Disease
7,8,"p(HGNC:""CLSTN1"")",NaN,NaN,p,NaN,NaN,HGNC,8,NaN,NaN,NaN,True,CLSTN1
8,9,"complex(p(HGNC:CLSTN1),p(HGNC:KLC1))",NaN,NaN,complex,True,NaN,NaN,9,NaN,NaN,NaN,NaN,NaN
9,10,"p(HGNC:""KLC1"")",NaN,NaN,p,NaN,NaN,HGNC,10,NaN,NaN,NaN,True,KLC1


In [25]:
for bel in df_short.BEL:
    print(bel) # read bel

a(CHEBI:"gamma-secretase inhibitor")
complex(SCOMP:"gamma Secretase Complex")
p(HGNC:"BACE1")
reaction(reactants(p(HGNC:APP)),products(a(ADO:"Abeta_42")))
p(HGNC:"APP")
a(ADO:"Abeta_42")
path(MESHD:"Alzheimer Disease")
p(HGNC:"CLSTN1")
complex(p(HGNC:CLSTN1),p(HGNC:KLC1))
p(HGNC:"KLC1")
bp(MESHPP:"Axonal Transport")
list(p(HGNC:PSEN1),p(HGNC:NCSTN),p(HGNC:APH1A),p(HGNC:PSEN2))
p(HGNC:"PSEN1")
p(HGNC:"NCSTN")
p(HGNC:"APH1A")
p(HGNC:"PSEN2")
list(a(ADO:"Abeta_42"),a(ADO:"Abeta_40"))
a(ADO:"Abeta_40")
a(CHEBI:"phenanthroline")
a(CHEBI:"ethylenediaminetetraacetic acid")
a(CHEBI:"ethylene glycol bis(2-aminoethyl)tetraacetic acid ")
a(CHEBI:"calcium(2+)")
a(CHEBI:"magnesium(2+)")
p(HGNC:"ARRB2")
p(HGNC:"GPR3")
complex(p(HGNC:ARRB2), p(HGNC:APH1A))
p(HGNC:"PLD1")
g(HGNC:"APP")
g(HGNC:"APP",sub(G,275341,C))
p(HGNC:"APP",sub(V,717,L))
p(HGNC:"PSEN2",sub(N,141,I))
g(HGNC:"APP",sub(G,717,C))
p(HGNC:"PSEN1",sub(E,280,A))
p(HGNC:"APOE")
p(HGNC:"APOE",sub(R,132,S))
g(HGNC:"CD44")
g(dbSNP:"rs187116")

In [26]:
for v in df_short.value:
    print(v) # read balue

gamma-secretase inhibitor
"gamma Secretase Complex"
BACE1
nan
APP
Abeta_42
Alzheimer Disease
CLSTN1
nan
KLC1
Axonal Transport
nan
PSEN1
NCSTN
APH1A
PSEN2
nan
Abeta_40
phenanthroline
ethylenediaminetetraacetic acid
ethylene glycol bis(2-aminoethyl)tetraacetic acid 
calcium(2+)
magnesium(2+)
ARRB2
GPR3
nan
PLD1
APP
APP
APP
PSEN2
APP
PSEN1
APOE
APOE
CD44
rs187116
NPHP1
rs10173717
CADPS2
rs3757536
GREM2
rs12129547
rs11542041
nan
iron(2+)
hydrogen peroxide
nan
copper(1+)
response to oxidative stress
mitochondrial calcium ion transport
mitochondrial calcium ion homeostasis
electron transport chain
superoxide
ATP
calcium ion homeostasis
nan
hydroxy group
isoprostane
BACE2
CASP4
microglial cell activation involved in immune response
Microglia
inflammatory response pathway
Neurogenic Inflammation
reactive oxygen species metabolic process
nitric oxide
neuron apoptotic process
TNF
IL1B
MAPT
"Calcineurin Complex"
BAD
BAD
cell death
CREB Family
CREB Family
nan
CDK5R1
CDK5
CAPN Family
MAPT
neuron ap

In [36]:
df['1-to1'] = df.function + "(" + df.namespace + ':"' + df.value + '")' # create new column
df.loc[:,['BEL','1-to1']].head()



,BEL,1-to1
node_id,,
1,"a(CHEBI:""gamma-secretase inhibitor"")","a(CHEBI:""gamma-secretase inhibitor"")"
2,"complex(SCOMP:""gamma Secretase Complex"")","complex(SCOMP:""""gamma Secretase Complex"""")"
3,"p(HGNC:""BACE1"")","p(HGNC:""BACE1"")"
4,"reaction(reactants(p(HGNC:APP)),products(a(ADO...",NaN
5,"p(HGNC:""APP"")","p(HGNC:""APP"")"


In [47]:
# create a list of columns(1-1) which can be used for the  in the main table because they are 1-to-1 and 
# a list of columns with more than value 
for bel in df.BEL:
    booleans = []
    #if BEL[bel]== ',':
        # booleans.append(True)
    if BEL[bel]==('list'):
         booleans.append(False)
    else:
        booleans.append(False)
    
#print(BEL, row.function, row.namespace, row.value)



IndexError: invalid index to scalar variable.

Let's create automatically the main table `gene` with `to_sql` method of `pandas.DataFrame`. Exclude the columns which

In [28]:
# load the json into a DataFrame as we did it before
# 1. all columns in the table named 'gene'
# 2. create dynamically first DataFrames and then
#    tables named as the columns with 1-to-many relation. Store also the foreign key in this DataFrame 
# trick : itearete all columns then create new data faram and save this frame in table

In [29]:
# Code 1

Check this MySQL website [Schema Object Names](http://dev.mysql.com/doc/refman/5.7/en/identifiers.html) for valid column names.

[0-9a-zA-Z$_] are allowed characters

In [30]:
# correct column names with regex( regular expressions)

Let's rename our columns in the dataframe

### 1-to1

### 1-to-many